# Filtrowanie danych przejazdów autobusowych (5-12 stycznia 2025)

Skrypt w Pythonie z biblioteką Polars wczytuje dane transportowe z pliku CSV (`df_for_modelling_v2.csv`) i filtruje przejazdy z okresu 5-12 stycznia 2025 na podstawie kolumny `date`. Przefiltrowane dane są zapisywane do pliku `przejazdy_5_12_stycznia_2025.csv`. Skrypt optymalizuje przetwarzanie dużych zbiorów danych i zawiera obsługę błędów. Zdefiniowany `schema_overrides` zapewnia poprawność typów (np. `Utf8` dla `line`, `Datetime` dla dat), unikając błędów parsowania. Ograniczenie do 7 dni zmniejsza rozmiar danych, ułatwiając testowanie modeli. Skrypt przygotowuje dane do dalszego przetwarzania. Obsługa błędów i optymalizacja pamięci (`low_memory=True`) zwiększają niezawodność dla dużych zbiorów danych.

In [ ]:
import polars as pl
from datetime import datetime

file_path = "/kaggle/input/bus-punctuality-dataset/df_for_modelling_v2.csv"

schema_overrides = {
    "date": pl.Datetime,
    "line": pl.Utf8,
    "task": pl.Utf8,
    "stop_seq": pl.Int64,
    "stop_name": pl.Utf8,
    "stop_id": pl.Int64,
    "scheduled_arrival": pl.Datetime,
    "scheduled_departure": pl.Datetime,
    "actual_arrival": pl.Datetime,
    "actual_departure": pl.Datetime,
    "detection_type": pl.Utf8,
    "delay": pl.Float64,
    "stop_desc": pl.Utf8,
    "stop_lat": pl.Float64,
    "stop_lon": pl.Float64,
    "is_weekday": pl.Boolean,
    "arrival_hour": pl.Int64,
    "is_holiday": pl.Boolean
}

try:
    print("Rozpoczynanie pracy")
    df = pl.read_csv(
        file_path,
        separator=";",
        null_values="NULL",
        schema_overrides=schema_overrides,
        try_parse_dates=True,
        low_memory=True
    )
    
    start_date = datetime(2025, 1, 5)
    end_date = datetime(2025, 1, 12, 23, 59, 59)

    filtered_df = df.filter(
        (pl.col("date").dt.date() >= start_date.date()) &
        (pl.col("date").dt.date() <= end_date.date())
    )
    
    print(f"Liczba przejazdów w okresie 5-12 stycznia 2025: {len(filtered_df)}")
    print("Kolumny w przefiltrowanym zbiorze danych:")
    print(filtered_df.columns)
    print("\nPierwsze 5 wierszy przefiltrowanych danych:")
    print(filtered_df.head(5))
    
    filtered_df.write_csv("przejazdy_5_12_stycznia_2025.csv", separator=";")

except FileNotFoundError:
    print(f"Plik {file_path} nie został znaleziony.")
except Exception as e:
    print(f"Wystąpił błąd podczas wczytywania lub filtrowania danych: {e}")

# Sprwdzenie liczby liniii autobusowych
Dla poprawnego dzialania sieci neuronowej potrzebujemy stworzyc osobny neuron dla kazdej linii, zgodnie z artykułem.

In [ ]:
import polars as pl

file_path = "../datasets/przejazdy_5_12_stycznia_2025.csv"

try:
    df = pl.read_csv(
        file_path,
        separator=";",
        null_values="NULL",
        schema_overrides={
            "date": pl.Datetime,
            "line": pl.Utf8,
            "task": pl.Utf8,
            "stop_seq": pl.Int64,
            "stop_name": pl.Utf8,
            "stop_id": pl.Int64,
            "scheduled_arrival": pl.Datetime,
            "scheduled_departure": pl.Datetime,
            "actual_arrival": pl.Datetime,
            "actual_departure": pl.Datetime,
            "detection_type": pl.Utf8,
            "delay": pl.Float64,
            "stop_desc": pl.Utf8,
            "stop_lat": pl.Float64,
            "stop_lon": pl.Float64,
            "is_weekday": pl.Boolean,
            "arrival_hour": pl.Int64,
            "is_holiday": pl.Boolean
        }
    )
    print(df["line"].unique().to_list())
    unique_lines = df["line"].unique().to_list()
    print(f"Liczba unikalnych linii: {len(unique_lines)}")



except FileNotFoundError:
    print(f"Plik {file_path} nie został znaleziony.")
except Exception as e:
    print(f"Wystąpił błąd podczas wczytywania pliku: {e}")

Wszystkie kolumny w pliku:
['10', '256', 'T8', '207', '178', '262', '267', '156', 'N8', '122', 'N4', '186', 'N5', '136', '123', '107', '8', '148', '154', '167', '112', 'N3', '124', '295', '120', '3', '249', '7', 'N14', '131', '113', '212', '232', '199', '149', '115', 'N9', '179', '4', '9', '258', '158', '208', '168', '6', '512', '162', '255', '127', '5', '117', 'N6', '269', '106', 'N78', '210', '213', '289', '174', 'N2', '130', '307', '283', '111', '200', '138', 'N1', '126', '205', '143', '166', 'N56', '268', '169', '12', '110', 'N16', '227', '195', '184', '175', '155', '157', '969', '132', '2', '118', '116', '171', '189', '108', '176', '100', '11']
Liczba unikalnych linii: 94


# Model sieci neuronowej

Model implementuje głęboką sieć neuronową (Fully Connected Neural Network, FCNN) do przewidywania czasu podróży (Trip Time) autobusu do następnego przystanku, opartą na metodologii z artykułu „Real-Time Bus Arrival Prediction: A Deep Learning Approach for Enhanced Urban Mobility”. 
Wykorzystuje one-hot encoding dla 94 unikalnych linii autobusowych, aproksymowaną odległość między przystankami, cechy czasowe (godziny szczytu, typ dnia) oraz numer przystanku, skalowane za pomocą MinMaxScaler. Architektura FCNN z pięcioma warstwami ukrytymi jest trenowana na GPU z użyciem 5-krotnej walidacji krzyżowej, aby zapewnić niezawodność predykcji na małym zbiorze danych (5-12 stycznia 2025). Model minimalizuje błąd RMSE, umożliwiając precyzyjne przewidywanie czasów przyjazdu, gotowe do integracji z aplikacjami mobilnymi.

In [ ]:
import polars as pl
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import KFold
from geopy.distance import geodesic

file_path = "../datasets/przejazdy_5_12_stycznia_2025.csv"
print("Rozpoczęto wczytywanie danych...")
df = pl.read_csv(
    file_path,
    separator=";",
    null_values="NULL",
    schema_overrides={
        "date": pl.Datetime,
        "line": pl.Utf8,
        "task": pl.Utf8,
        "stop_seq": pl.Int64,
        "stop_name": pl.Utf8,
        "stop_id": pl.Int64,
        "scheduled_arrival": pl.Datetime,
        "scheduled_departure": pl.Datetime,
        "actual_arrival": pl.Datetime,
        "actual_departure": pl.Datetime,
        "detection_type": pl.Utf8,
        "delay": pl.Float64,
        "stop_desc": pl.Utf8,
        "stop_lat": pl.Float64,
        "stop_lon": pl.Float64,
        "is_weekday": pl.Boolean,
        "arrival_hour": pl.Int64,
        "is_holiday": pl.Boolean
    }
)
print("Dane wczytane pomyślnie.")

print("Obliczanie Trip Time...")
df = df.with_columns(
    (pl.col("actual_arrival") - pl.col("date")).dt.total_seconds().alias("trip_time")
)
print("Trip Time obliczony.")

print("Obliczanie odległości między przystankami...")
df = df.sort(["line", "task", "stop_seq"])
df = df.with_columns(
    pl.col("stop_lat").shift(-1).alias("next_stop_lat"),
    pl.col("stop_lon").shift(-1).alias("next_stop_lon")
)
df = df.with_columns(
    pl.when(pl.col("next_stop_lat").is_not_null())
    .then(
        pl.struct(["stop_lat", "stop_lon", "next_stop_lat", "next_stop_lon"])
        .map_elements(
            lambda x: geodesic(
                (x["stop_lat"], x["stop_lon"]),
                (x["next_stop_lat"], x["next_stop_lon"])
            ).meters,
            return_dtype=pl.Float64
        )
    )
    .otherwise(None)
    .alias("distance")
)
print("Odległości obliczone.")

print("Dodawanie cechy rush_hour...")
df = df.with_columns(
    pl.col("arrival_hour").is_in([6, 7, 8, 9, 15, 16, 17, 18]).cast(pl.Int8).alias("rush_hour")
)
print("Cechą rush_hour dodana.")

print("Dodawanie cechy far_status...")
df = df.with_columns(
    (pl.col("distance") > 250).cast(pl.Int8).alias("far_status")
)
print("Cecha far_status dodana.")

print("Usuwanie brakujących danych...")
df = df.drop_nulls()
print("Brakujące dane usunięte.")

print("Kodowanie one-hot dla linii...")
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
line_encoded = encoder.fit_transform(df[["line"]].to_numpy())
line_encoded_cols = [f"line_{val}" for val in encoder.categories_[0]]
line_encoded_df = pl.DataFrame(line_encoded, schema=line_encoded_cols)
print(f"Kodowanie one-hot zakończone: {len(line_encoded_cols)} linii.")

print("Przygotowanie cech wejściowych...")
features = line_encoded_cols + ["distance", "is_weekday", "rush_hour", "stop_seq", "far_status"]
X = pl.concat([line_encoded_df, df.select(["distance", "is_weekday", "rush_hour", "stop_seq", "far_status"])], how="horizontal").to_numpy()
y = df["trip_time"].to_numpy()
print("Cechy wejściowe przygotowane.")

print("Skalowanie cech...")
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
print("Cechy przeskalowane.")

class FCNN(nn.Module):
    def __init__(self, input_size):
        super(FCNN, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 320),
            nn.ReLU(),
            nn.Linear(320, 200),
            nn.ReLU(),
            nn.Linear(200, 100),
            nn.ReLU(),
            nn.Linear(100, 40),
            nn.ReLU(),
            nn.Linear(40, 5),
            nn.ReLU(),
            nn.Linear(5, 1)
        )
    
    def forward(self, x):
        return self.layers(x)

print("Inicjalizacja walidacji krzyżowej...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
kf = KFold(n_splits=5, shuffle=True, random_state=42)
rmse_scores = []
print(f"Walidacja krzyżowa zainicjalizowana (5 foldów), urządzenie: {device}.")

for fold, (train_idx, val_idx) in enumerate(kf.split(X_scaled)):
    print(f"\nRozpoczęto fold {fold+1}...")
    X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]
    
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1).to(device)
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
    y_val_tensor = torch.tensor(y_val, dtype=torch.float32).reshape(-1, 1).to(device)
    
    model = FCNN(input_size=X_scaled.shape[1]).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    
    num_epochs = 50
    batch_size = 64
    print(f"Trening folda {fold+1} rozpoczęty (50 epok)...")
    for epoch in range(num_epochs):
        model.train()
        for i in range(0, len(X_train), batch_size):
            batch_X = X_train_tensor[i:i+batch_size]
            batch_y = y_train_tensor[i:i+batch_size]
            
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
    
    print(f"Trening folda {fold+1} zakończony. Ewaluacja...")
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val_tensor)
        mse = criterion(val_outputs, y_val_tensor).item()
        rmse = np.sqrt(mse)
        rmse_scores.append(rmse)
        print(f"Fold {fold+1}, RMSE: {rmse:.2f} seconds")

print("\nWszystkie foldy zakończone.")
print(f"Średnie RMSE z walidacji krzyżowej: {np.mean(rmse_scores):.2f} ± {np.std(rmse_scores):.2f} seconds")

Rozpoczęto wczytywanie danych...
Dane wczytane pomyślnie.
Obliczanie Trip Time...
Trip Time obliczony.
Obliczanie odległości między przystankami...
Odległości obliczone.
Dodawanie cechy rush_hour...
Cechą rush_hour dodana.
Dodawanie cechy far_status...
Cecha far_status dodana.
Usuwanie brakujących danych...
Brakujące dane usunięte.
Kodowanie one-hot dla linii...
Kodowanie one-hot zakończone: 94 linii.
Przygotowanie cech wejściowych...
Cechy wejściowe przygotowane.
Skalowanie cech...
Cechy przeskalowane.
Inicjalizacja walidacji krzyżowej...
Walidacja krzyżowa zainicjalizowana (5 foldów), urządzenie: cpu.

Rozpoczęto fold 1...
Trening folda 1 rozpoczęty (50 epok)...


KeyboardInterrupt: 